# **How to paraphrase text using transformers in Python**

Chanin Nantasenamat

[Data Professor](http://youtube.com/dataprofessor), http://youtube.com/dataprofessor

**Notes and References:**
- PEGASUS is an acronym for Pre-training with Extracted Gap-sentences for Abstractive SUmmarization Sequence-to-sequence models
- [PEGASUS](https://huggingface.co/tuner007/pegasus_paraphrase) model used herein is from *Huggingface's* **transformers** library
- [PEGASUS model from Google Research](https://github.com/google-research/pegasus)
- Read the original paper [PEGASUS: Pre-training with Extracted Gap-sentences for Abstractive Summarization](https://arxiv.org/abs/1912.08777)
- Check out the book [Transformers for Natural Language Processing](https://amzn.to/39IC6E6)


# **Install library**

In [ ]:
! pip install sentence-splitter

In [ ]:
! pip install transformers

In [ ]:
! pip install SentencePiece

In [ ]:
# https://huggingface.co/tuner007/pegasus_paraphrase

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer.prepare_seq2seq_batch([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

---

## **Processing a single sentence**

In [ ]:
text = "In this video, I will be showing you how to build a stock price web application in Python using the Streamlit and yfinance library."

In [ ]:
get_response(text, 5)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3221: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


['In this video, I will show you how to use the Streamlit and yfinance libraries to build a stock price web application.',
 'In this video, I will show you how to build a stock price web application in Python using the Streamlit and yfinance libraries.',
 'In this video, I will show you how to build a stock price web application using the Streamlit and yfinance libraries.',
 'In this video, I will show you how to use the Streamlit and yfinance libraries to build a stock price web application in Python.',
 'In this video, I will show you how to use the Streamlit and yfinance library to build a stock price web application.']

In [ ]:
get_response(text, 1)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3221: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


['In this video, I will show you how to use the Streamlit and yfinance libraries to build a stock price web application.']

## **Processing a paragraph of text**

In [ ]:
# Paragraph of text
context = "In this video, I will be showing you how to build a stock price web application in Python using the Streamlit and yfinance library. The app will be able to retrieve company information as well as the stock price data for S and P 500 companies. All of this in less than 50 lines of code."
print(context)

In this video, I will be showing you how to build a stock price web application in Python using the Streamlit and yfinance library. The app will be able to retrieve company information as well as the stock price data for S and P 500 companies. All of this in less than 50 lines of code.


In [ ]:
# Takes the input paragraph and splits it into a list of sentences
from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')

sentence_list = splitter.split(context)
sentence_list

['In this video, I will be showing you how to build a stock price web application in Python using the Streamlit and yfinance library.',
 'The app will be able to retrieve company information as well as the stock price data for S and P 500 companies.',
 'All of this in less than 50 lines of code.']

In [ ]:
# Do a for loop to iterate through the list of sentences and paraphrase each sentence in the iteration
paraphrase = []

for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3221: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


In [ ]:
# This is the paraphrased text
paraphrase

[['In this video, I will show you how to use the Streamlit and yfinance libraries to build a stock price web application.'],
 ['The stock price data for S and P 500 companies will be retrieved by the app.'],
 ['This is in less than 50 lines of code.']]

In [ ]:
paraphrase2 = [' '.join(x) for x in paraphrase]
paraphrase2

['In this video, I will show you how to use the Streamlit and yfinance libraries to build a stock price web application.',
 'The stock price data for S and P 500 companies will be retrieved by the app.',
 'This is in less than 50 lines of code.']

In [ ]:
# Combines the above list into a paragraph
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
paraphrased_text

'In this video, I will show you how to use the Streamlit and yfinance libraries to build a stock price web application. The stock price data for S and P 500 companies will be retrieved by the app. This is in less than 50 lines of code.'

In [ ]:
# Comparison of the original (context variable) and the paraphrased version (paraphrase3 variable)

print(context)
print(paraphrased_text)

In this video, I will be showing you how to build a stock price web application in Python using the Streamlit and yfinance library. The app will be able to retrieve company information as well as the stock price data for S and P 500 companies. All of this in less than 50 lines of code.
In this video, I will show you how to use the Streamlit and yfinance libraries to build a stock price web application. The stock price data for S and P 500 companies will be retrieved by the app. This is in less than 50 lines of code.
